# read shape files

In [1]:
import geopandas as gpd
soa_shape_map_path = '/home/mahdi/data/PRISM_tmin_stable_4kmD2_20160612_nc/doc_2024-03-26_15-21-36.gridpoint_2000.shp/.gridpoint_2000.shp.shp'
sub_basin_path = '/home/mahdi/data/PRISM_tmin_stable_4kmD2_20160612_nc/central_valley_3857/central_valley_3857.shp'

mesh_grid_path = "/home/mahdi/data/PRISM_tmin_stable_4kmD2_20160612_nc/prism_4km_mesh/prism_4km_mesh.shp"

# soa_shape_map = gpd.read_file(soa_shape_map_path)
# sub_basin_map = gpd.read_file(sub_basin_path)

mesh = gpd.read_file(mesh_grid_path)

# soa_shape_map_geo = soa_shape_map.to_crs(epsg=4326)
# sub_basin_map_geo = sub_basin_map.to_crs(epsg=4326)
# mesh_geo = mesh.to_crs(epsg=4326)

# sub_basin_1 = sub_basin_map_geo[sub_basin_map_geo['Subregion_'] == 1]

mesh


,col,row,lon,lat,geometry
0,717,12,-95.125000,49.416667,"POLYGON ((-95.14583 49.43750, -95.10417 49.437..."
1,716,13,-95.166667,49.375000,"POLYGON ((-95.18750 49.39583, -95.14583 49.395..."
2,717,13,-95.125000,49.375000,"POLYGON ((-95.14583 49.39583, -95.10417 49.395..."
3,718,13,-95.083333,49.375000,"POLYGON ((-95.10417 49.39583, -95.06250 49.395..."
4,719,13,-95.041667,49.375000,"POLYGON ((-95.06250 49.39583, -95.02083 49.395..."
...,...,...,...,...,...
481626,1040,609,-81.666667,24.541667,"POLYGON ((-81.68750 24.56250, -81.64583 24.562..."
481627,1041,609,-81.625000,24.541667,"POLYGON ((-81.64583 24.56250, -81.60417 24.562..."
481628,1028,610,-82.166667,24.500000,"POLYGON ((-82.18750 24.52083, -82.14583 24.520..."
481629,1029,610,-82.125000,24.500000,"POLYGON ((-82.14583 24.52083, -82.10417 24.520..."


## sample

In [7]:
mesh_center = mesh[(mesh['col'] == 71) & (mesh['row'] == 220)]
mesh_up = mesh[(mesh['col'] == 71) & (mesh['row'] == 219)]
mesh_down = mesh[(mesh['col'] == 71) & (mesh['row'] == 221)]
mesh_right = mesh[(mesh['col'] == 72) & (mesh['row'] == 220)]
mesh_left = mesh[(mesh['col'] == 70) & (mesh['row'] == 220)]

## create a point layer and list of coordinates

In [24]:
from shapely.geometry import Point

# Create a list of points with latitude and longitude
points = [
    Point(lon, lat) for lon, lat in [
        (mesh_center["lon"].item(), mesh_center["lat"].item()),
        (mesh_up["lon"].item(), mesh_up["lat"].item()),
        (mesh_down["lon"].item(), mesh_down["lat"].item()),
        (mesh_right["lon"].item(), mesh_right["lat"].item()),
        (mesh_left["lon"].item(), mesh_left["lat"].item())
        # Add more points as needed
    ]
]

# Create a GeoDataFrame from the list of points
shape_four = gpd.GeoDataFrame(geometry=points)

lat_lon_tuples_shape_four = list(zip(shape_four.geometry.y, shape_four.geometry.x))
lat_lon_tuples_shape_four

[(40.7500000000013, -122.04166666666643),
 (40.791666666667965, -122.04166666666643),
 (40.70833333333463, -122.04166666666643),
 (40.7500000000013, -121.99999999999974),
 (40.7500000000013, -122.08333333333309)]

In [ ]:
# clipped_points = gpd.clip(soa_shape_map_geo, sub_basin_1)

# lat_lon_tuples = list(zip(clipped_points.geometry.y, clipped_points.geometry.x))


# generate time series

In [25]:
import datetime

def generate_time_series(start_date, end_date, freq):
    """
    Generate a time series of dates in the format "YYYYMMDD".

    Args:
        start_date (str): The start date in the format "YYYY-MM-DD".
        end_date (str): The end date in the format "YYYY-MM-DD".
        freq (str): The frequency of the time series. Possible values are
            "D" (daily), "W" (weekly), "M" (monthly), etc.

    Returns:
        list: A list of dates in the format "YYYYMMDD".
    """
    date_series = []
    start = datetime.datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.datetime.strptime(end_date, "%Y-%m-%d")
    current_date = start

    while current_date <= end:
        date_series.append(current_date.strftime("%Y%m%d"))
        current_date += datetime.timedelta(days=1) if freq == "D" else datetime.timedelta(weeks=1) if freq == "W" else datetime.timedelta(days=30.4375)

    return date_series

# function of download prism data

In [26]:
import requests
import pandas as pd

def download_file(url):
  """Downloads a file from the given URL and saves it locally."""
  local_filename = url.split('/')[-1]
  # NOTE the stream=True parameter below
  with requests.get(url, stream=True) as r:
    r.raise_for_status()
    with open(local_filename, 'wb') as f:
      for chunk in r.iter_content(chunk_size=8192): 
        f.write(chunk)
  return local_filename


def download_prism_data(lat_lon_tuples, output_folder):
  """Downloads PIRISM data for a list of latitude and longitude tuples.

  Args:
    lat_lon_tuples: A list of tuples containing (latitude, longitude) coordinates.
    output_folder: The folder where downloaded data files will be saved.
  """
  session = requests.Session()
  base_url = "https://prism.oregonstate.edu/explorer/dataexplorer/rpc.php"
  data = {
    "spares": "4km",
    "interp": "0",
    "stats": "ppt tmin tmean tmax",  # You can customize the variables here
    "units": "si",
    "range": "daily",
    "start": "19910101",
    "end": "19911231",
    "stability": "stable",
    "call": "pp/daily_timeseries",
    "proc": "gridserv"
  }
  headers = {
    "Accept": "application/json, text/javascript, */*; q=0.01",
    "Accept-Encoding": "gzip, deflate, br",
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:123.0) Gecko/20100101 Firefox/123.0",
    "DNT": "1",
    "Sec-GPC": "1",
  }

  for lat, lon in lat_lon_tuples:
    data["lat"] = str(lat)
    data["lon"] = str(lon)
    response = session.post(base_url, data=data, headers=headers)
    res_json = response.json()

    
    download_url = "https://prism.oregonstate.edu/explorer/tmp/" + res_json["result"]["csv"]
    filename = f"{output_folder}/{lat}_{lon}.csv"  # Customize filename format
    download_file(download_url)
    print(f"Downloaded data for ({lat}, {lon}) to {filename}")
    


## example

In [27]:
# Example usage
output_folder = "/home/mahdi/data/export"  # Change this if needed
download_prism_data(lat_lon_tuples_shape_four, output_folder)

Downloaded data for (40.7500000000013, -122.04166666666643) to /home/mahdi/data/export/40.7500000000013_-122.04166666666643.csv
Downloaded data for (40.791666666667965, -122.04166666666643) to /home/mahdi/data/export/40.791666666667965_-122.04166666666643.csv
Downloaded data for (40.70833333333463, -122.04166666666643) to /home/mahdi/data/export/40.70833333333463_-122.04166666666643.csv
Downloaded data for (40.7500000000013, -121.99999999999974) to /home/mahdi/data/export/40.7500000000013_-121.99999999999974.csv
Downloaded data for (40.7500000000013, -122.08333333333309) to /home/mahdi/data/export/40.7500000000013_-122.08333333333309.csv


# use pyPRISMClimate library and FTP

In [ ]:
import os
import sys
sys.path.append('/home/mahdi/data/pyPRISMClimate')
from pyPRISMClimate.quick_tools import *

## download daily data

In [ ]:
data = get_prism_dailys(
    variable ="tmean",
    min_date="1991-01-01",
    max_date="2020-12-31",
    dest_path="/home/mahdi/data/export"
)

### open bil file

In [ ]:
from osgeo import gdal

# Open the BIL file
dataset = gdal.Open("/home/mahdi/data/export/PRISM_tmean_stable_4kmD2_20201227_bil.bil")

### convert to array

In [ ]:
# Get the number of bands and data type
num_bands = dataset.RasterCount
data_type = dataset.GetRasterBand(1).DataType
# Read the data as a NumPy array
data = dataset.GetRasterBand(1).ReadAsArray()
# (Optional) Get geotransform information
geotransform = dataset.GetGeoTransform()
geotransform